In [5]:
!rm -rf .data/Pokemon_LMDB

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
import sys
import lmdb
import pickle
import numpy as np


In [3]:
os.chdir('../') # adress to git dir
print(os.getcwd())

/home/azeghost/git/lmdb_new/LMDB_Datasets


In [4]:
# from tensorflow.keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
# from transformation.file_utils import get_file_path
# from transformation.file_image_generator import create_image_lists

In [5]:
from transformation.lmdb_transformer import LmdbTransformer

In [6]:
# Store images in lmdb

In [7]:
images_dir = '/home/azeghost/git/Generative_Models/data/.pokemon/' #Folder to images
validation_percentage = 30
valid_format = 'png'

In [8]:
transformer = LmdbTransformer(images_dir, validation_percentage, valid_format)

ValueError: Image directory png not found.

In [10]:
from datasets_customed.label_by_filename import get_label_by_filename

In [12]:
import os
import numpy as np


def get_label_by_filename(img_path):
    name, _ = os.path.splitext(img_path)
    vid_img_arr = name.split(sep=os.sep)[-1:]
    return {'label1': np.array(vid_img_arr), 'label2': np.array(vid_img_arr), 'label3': np.array(vid_img_arr)}

In [13]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = '.data/Pokemon_LMDB'
           ,category='training',target_size=None,color_mode='rgb')

Storing 591.data/Pokemon_LMDB/_training


In [14]:
transformer.transform_store(labels_fn=get_label_by_filename,image_dir=images_dir, lmdb_dir = '.data/Pokemon_LMDB'
           ,category='validation',target_size=None,color_mode='rgb')

Storing 218.data/Pokemon_LMDB/_validation


# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [11]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

            for label in labels_list:
                # _lab = {label: eval(f'dataset.{label}')}
                # labels = {**labels, **_lab}
                if label in labels:
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels = {label: [eval(f'dataset.{label}')] }
                
    env.close()
    return {'images': images, **labels}

In [12]:
ds = read_many_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 591)


In [13]:
ds.keys()

dict_keys(['images', 'label3'])

In [14]:
ds['images'][0].shape

(400, 400, 3)

In [15]:
ds['label1'][0], ds['label2'][0], ds['label3'][0], 

KeyError: 'label1'

## LMDBImageIterator and LMDBImageGenerator

In [20]:
os.chdir('/home/azeghost/git/Generative_Models/')

In [21]:
os.getcwd()

'/home/azeghost/git/Generative_Models'

In [11]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class LMDBImageIterator(Iterator):


    def __init__(self,
                 num_images,
                 category,
                 lmdb_dir,
                 batch_size,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx(),
                 ):
        
        self.category = category
        self.batch_size = batch_size
        
        
        self.lmdb_dir = lmdb_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format
        print("Initializing Iterator " + category +" Number of images " +str(num_images))
        print(category,lmdb_dir, batch_size,shuffle,seed)
        self.env = lmdb.open(lmdb_dir, readonly=True)
        
        Iterator.__init__(self, num_images, batch_size, shuffle, seed)
        
        
    def __del__(self):
        self.env.close()
        
        
    def _get_batches_of_transformed_samples(self, index_array):
        print(index_array)
        images, labels = [], []
        
        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        else:
                with self.env.begin() as txn:
                    for image_id in index_array:
                        data = txn.get(f"{image_id:08}".encode("ascii"))
                        dataset = pickle.loads(data)
                        images.append(dataset.get_image())
                        labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]

                        for label in labels_list:
                            _lab = {label: eval(f'dataset.{label}')}
                            labels = {**labels, **_lab}
        return {'images': images, **labels}


In [12]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message


class LMDBImageGenerator(ImageDataGenerator):
    def flow_from_lmdb_lists(self, 
                              num_images,
                              category,
                              lmdb_dir,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              shuffle =True,
                              seed=None
                              ):

        
          

        return LMDBImageIterator(
                             num_images = num_images,
                             category = category,
                             lmdb_dir = lmdb_dir,
                             batch_size  = batch_size,
                             episode_len = episode_len,
                             episode_shift =episode_shift,
                             shuffle = shuffle,
                             seed = seed)


def get_generators( val_lmdb_dir, val_num_images, tra_lmdb_dir, tra_num_images, 
                   batch_size, episode_len=None, episode_shift=None):

    train_datagen = LMDBImageGenerator()

    valid_datagen = LMDBImageGenerator()

    train_generator = train_datagen.flow_from_lmdb_lists(
        num_images = tra_num_images,
        category='training',
        lmdb_dir=tra_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_lmdb_lists(
        num_images = val_num_images,
        category='validation',
        lmdb_dir=val_lmdb_dir,
        batch_size=batch_size,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator



In [13]:
training_generator, testing_generator = get_generators(
    val_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation',
    val_num_images = 218,
    tra_lmdb_dir = '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training', 
    tra_num_images = 591,
    batch_size=100
)

Initializing Iterator training Number of images 591
training /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_training 100 True 0
Initializing Iterator validation Number of images 218
validation /home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Pokemon_LMDB/_validation 100 True 0


In [14]:
x, y = training_generator.next()

[225  14  85 418 132 213 186 340 122 394 229 586 261 469  52 440 162 173
 462 515 468 315  21 188 403 103 330 230  92 272 483 415 240 202 235 334
 375 528 157 127  64 463  37 250 453  34 343 283 172   1 366 262  48 506
 402 245 329 407 187 200  12 358 165 479 588 400  31 439 532 319 313 347
 397 466 420  15 509 546 243 332 427  75  10 242 535 561 278 441 589 574
 514 386 457 249 170  55  90 460 560 518]


TypeError: 'list' object is not a mapping

In [ ]:
import os
os.chdir('/home/azeghost/git/LMDB_Datasets')

In [ ]:
#Wrapper class for dataset
class Pokemon_Image:
    def __init__(self, image, label):
        self.channels = image.shape[2]
        self.size = image.shape[:2]
        self.image = image.tobytes()
        self.label = label #additional data to be stored (make it string)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.float32) #pay attention if you  don't use create_image_lists
        return images.reshape(*self.size, self.channels)     #then dtype will be different

In [ ]:
def store_single_lmdb(filename, img , index, label, num_images):
    """ Stores a wrapper to LMDB.
    """
    map_size = num_images * img.nbytes * 10
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = Pokemon_Image(img, label)
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))

    env.close()

In [ ]:
def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    print('Storing '+ str(num_images)+lmdb_dir+os.sep+'_{}'.format(category))
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]     
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        
        store_single_lmdb(index = index, filename=name, img = img,label = y ,num_images = num_images)

In [ ]:
class LMDB_transformer:
    def __init(image_lists**, )
        create_img_list
    
    def transform_store():
        store_lmdb()

In [ ]:
!rm -rf ./LMDB/
!mkdir ./LMDB/
# save_to_dir=None,
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='training',target_size=None,color_mode='rgb',save_prefix='',save_format='png')
store_lmdb(image_lists, image_dir=images_dir, lmdb_dir = './LMDB'
           ,category='validation',target_size=None,color_mode='rgb',save_prefix='',save_format='png')

# Done on ubuntu system is local machine does not have tar 
 - Install git bash then u can use such commands.
 - Go to windows store and download Ubuntu emulator wich just installs bash
 - On colab these should work

 ## For Training

In [ ]:
!tar -cvjf pokemon_training.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_training/* 

In [ ]:
!split -b 10M pokemon_training.tar.bz2 "pokemon_training.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [ ]:
#Check if they are created
!ls -lh pokemon_training.tar.part*

In [ ]:
#move to correct folder and push to git
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_training.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon 

In [ ]:
# push to git
# !cd /home/azeghost/git/LMDB_Datasets
# !git add ../pokemon/*
# # !git status
# !git commit -m "pokemon_training"
# !git push orign master # Check if you have ssh token enabled for bash use otherwise enter login and password

## For validation

In [ ]:
!tar -cvjf pokemon_validation.tar.bz2 /home/azeghost/git/Generative_Models/LMDB/_validation/* 

In [ ]:
!split -b 200M pokemon_training.tar.bz2 "pokemon_validation.tar.part"
!ls -lh pokemon_validation.tar.part*

In [ ]:
!mkdir /home/azeghost/git/LMDB_Datasets/pokemon
!mv pokemon_validation.tar.part* /home/azeghost/git/LMDB_Datasets/pokemon
!ls /home/azeghost/git/LMDB_Datasets/pokemon

In [ ]:
# !cd /home/azeghost/git/LMDB_Datasets
# !git add pokemon/*
# !git status
# !git commit -m "pokemon_validation"
# !git push orign master

# After downloading from Git combine and unzip

## combine them

In [ ]:
#pokemon_training
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_training.tar.bz2
#!cat <split files put * at the end> > <final zip name>
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

In [ ]:
#pokemon_validation
!cat /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.part* >/home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2
!ls -la /home/azeghost/git/LMDB_Datasets/pokemon

## Unzip   

In [ ]:
!tar -xf /home/azeghost/git/LMDB_Datasets/pokemon/pokemon_validation.tar.bz2 -C /home/azeghost/git/LMDB_Datasets
!ls -la /home/azeghost/git/LMDB_Datasets

# Read the lmdb 

In [33]:
from transformation.wrappers import DatasetWrapper

In [3]:
def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], []
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            pokemon_image = pickle.loads(data)
            images.append(pokemon_image.get_image())
            labels.append(pokemon_image.label)
    env.close()
    return images, labels

In [4]:
x, y = read_many_lmdb('/home/azeghost/PycharmProjects/LMDB_DATASETS/.data/Pokemon_LMDB/_training', 591)
print(len(x), len(y))

TypeError: a bytes-like object is required, not 'NoneType'